In [1]:
from keras.models import load_model
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix #classification_report
import itertools  # for confusion matrix plot
from keras.utils import np_utils
%matplotlib inline


import cv2
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))      
class_label = ['background', 'body','nose', 'tail']
n_class = len(class_label)
target_size = 40
## model load



C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:

model_path = '/Data/mice_processed/ak_mice.h5'
model=load_model(model_path)

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 40, 40, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 40, 40, 64)        0         
_________________________________________________________________
activation_5 (Activation)    (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 64)        102464    
__________

In [6]:
# transfer learning
count = 0
for layer in model.layers[0:17]:
    count+=1
    layer.trainable = False
print(count)
model.summary()

17
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 40, 40, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 40, 40, 64)        0         
_________________________________________________________________
activation_5 (Activation)    (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 64)        102464    
_______

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [10]:
from keras.models import load_model
from keras.utils import np_utils
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

#import plot as pl

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))      
class_label = ['background', 'body','nose', 'tail']
n_class = len(class_label)
target_size = 40
epochs = 50
batch_size = 16
data_path ='/Data/mice_split'    
    

def preprocess_img(img):    
    img=img.astype(np.uint8) # this is to match keras type? but uint8 might be more universal
    img = clahe.apply(img) 
    img = img/255.  # normalize
    img = np.reshape(img, (target_size,target_size,1))    
    return img


early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0.001, patience = 20, 
                               verbose = 1, mode = 'max')

## Fit generator with augmentation

def gen_train(model, data_path=data_path):     
    train_datagen = ImageDataGenerator(
               
                rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
                # randomly shift images horizontally (fraction of total width)
                # set mode for filling points outside the input boundaries
                fill_mode='nearest',
                cval=0.,  # value used for fill_mode = "constant"
                horizontal_flip=True,  # randomly flip images
                vertical_flip=True,  # randomly flip images
                # set rescaling factor (applied before any other transformation)
                #rescale=1./255,
                # set function that will be applied on each input
                preprocessing_function=preprocess_img,
                # image data format, either "channels_first" or "channels_last"
                data_format=None, 
                # fraction of images reserved for validation (strictly between 0 and 1)
                validation_split=0)
        
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_img)#rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
            data_path+'/train/', shuffle=True,
            target_size=(target_size, target_size),
            #batch_size=batch_size, 
            color_mode = 'grayscale',
            class_mode='categorical') #,  save_to_dir = data_path+'aug', save_prefix='aug_',) # option is for monitoring data augmentation
    
    val_generator = test_datagen.flow_from_directory(
            data_path+'/validation/', shuffle=True,
            target_size=(target_size, target_size),
            #batch_size=batch_size, 
            color_mode = 'grayscale',
            class_mode='categorical') #,  save_to_dir = data_path+'aug', save_prefix='aug_',) # option is for monitoring data augmentation
    
    test_generator = test_datagen.flow_from_directory(
            data_path+'/test/', shuffle=False,
            target_size=(target_size, target_size),
            #batch_size=batch_size, 
            color_mode = 'grayscale',
            class_mode='categorical') #,  save_to_dir = data_path+'aug', save_prefix='aug_',) # option is for monitoring data augmentation
    
    
    hist = model.fit_generator(
            train_generator,
            #steps_per_epoch=30,
            epochs=epochs, verbose=1,
            validation_data=val_generator,
            #validation_steps=10,
            callbacks = [early_stopping])
    
    return hist, test_generator

### evaluation result for generator / flow from dir
def gen_eval(model, test_gen):
    pred = model.predict_generator(test_gen)
    
    y_true = test_gen.classes
    y_pred = np.argmax(pred,axis=1)

    confusion_result = confusion_matrix(y_true, y_pred)
    pl.plot_confusion_matrix(confusion_result, classes = class_label, normalize = True, title = 'Confusion_matrix')

In [13]:

model.compile(loss = categorical_crossentropy,
              optimizer = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7),
              metrics = ['accuracy'])

hist, test_gen = gen_train(model, data_path)
    

Found 288 images belonging to 4 classes.
Found 96 images belonging to 4 classes.
Found 96 images belonging to 4 classes.
Epoch 1/50
9/9 [==============================] - 6s 662ms/step - loss: 0.4100 - acc: 0.8715 - val_loss: 0.3871 - val_acc: 0.8958
Epoch 2/50
9/9 [==============================] - 6s 613ms/step - loss: 0.3606 - acc: 0.8438 - val_loss: 0.4118 - val_acc: 0.8854
Epoch 3/50
9/9 [==============================] - 5s 590ms/step - loss: 0.4022 - acc: 0.8264 - val_loss: 0.4105 - val_acc: 0.8854
Epoch 4/50
9/9 [==============================] - 5s 583ms/step - loss: 0.4196 - acc: 0.8264 - val_loss: 0.4089 - val_acc: 0.8854
Epoch 5/50
9/9 [==============================] - 5s 597ms/step - loss: 0.3466 - acc: 0.8715 - val_loss: 0.4113 - val_acc: 0.8854
Epoch 6/50
9/9 [==============================] - 5s 579ms/step - loss: 0.3869 - acc: 0.8333 - val_loss: 0.4109 - val_acc: 0.8854
Epoch 7/50
9/9 [==============================] - 5s 585ms/step - loss: 0.3535 - acc: 0.8507 - val_

In [62]:
np.random.randint(0,3)


1

In [12]:
model.save('mice_transfer.h5')
os.getcwd()

'F:\\github\\test'